In [69]:
import numpy as np
import math  

Решение Закировой Марфы, Б06-906а
# <center> Задача
Oпределить значение $f(0.95)$ :
1. с помощью интерполяции полиномами
2. с помоцью интерполяции сплайнами

\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline f(x) & 0,000 & 0,033 & 0,067 & 0,100 & 0,133 & 0,166 & 0,199 & 0,231 & 0,264 & 0,296 & 0,327 \\
\hline x & 0 & 0,1 & 0,2 & 0,3 & 0,4 & 0,5 & 0,6 & 0,7 & 0,8 & 0,9 & 1 \\
\hline
\end{array}

Оценить ошибку, используя то, что $\left|f^{(x)}(x)\right|<10^ 6 \text { и }\left|f^{IV}(x)\right|<0.1$  на отрезке интерполяции


__Интерполяция полиномами__


In [108]:
x = np.linspace(0,1, 11)
f = np.array([0, 0.033, 0.067, 0.1, 0.133, 0.166, 0.199, 0.231, 0.264, 0.296, 0.327])
z = 0.95

N = x.shape[0] - 1
print(f'число узлов(N+1): {N+1}')

число узлов(N+1): 11


Интерполяционным полиномом в форме Чебышева не получается, исследуемые значения функции монотонно возрастают, полученное значение меньше ближайшего следующего значения f(x_k)


Я поняла в чем ошибка, я использовала реккурентную формулу не для вывода конкретного полинома Чебышева, а для подсчета его значения в конкретных точках.

In [50]:
N = x.shape[0] - 1
a = []
f_095 = 0

#T = np.array([np.cos(j*np.arccos(x)) for j in range(N + 1)]) проблемы с арккосинусом, поэтому не работает так
T = {0: [[] for _ in range(N+1)], z :[1]}
T[0][0] = [1]*(N+1)

T[0][1] = x
T[z].append(z)


for i in range(2, N+1):
    T[0][i] = 2* T[0][i - 1] * x - T[0][i - 2]
    T[z].append(2* T[z][i - 1] * z - T[z][i - 2])
    
T[0][0] = [1/np.sqrt(2) for _ in range(N+1)]
T[z][0] = (1/np.sqrt(2))

for j in range(N+1):
    a.append((2/(N + 1))*np.sum(f * T[0][j]))
    f_095 += a[j] * T[z][j]

print(f_095)

    
    

0.1360102051668148


Через полином Лагранжа получилось:

In [68]:
fi = [1]*(N + 1)
for i in range(N + 1):
    for j in range(N + 1): 
        if i != j:
            fi[i] *= (z-x[j])/(x[i]-x[j])
fi = np.array(fi)
res = np.sum(fi*f)
print(f'f({z}): {np.round(res,3)}')


f(0.95): 0.309


$$\Delta  = \Delta_{\text{метода}} + \Delta_{\text{вычисл}}$$

In [116]:
delta_metod = (10**6) * (h**(N+1))/(N + 1)
delta_v = (2**(N+1)) * 0.001/(np.e * N * np.log(N))
print(f'delta_метода {delta_metod}')
print(f'delta: {np.round(delta_metod + delta_v, 3)}')

delta_метода 9.090909090909097e-07
delta: 0.033


__Интерполяция сплайнами__

In [100]:
h = 0.1
A = [[0]*(N-1) for _ in range(N-1)]

u_2 = [(f[i + 1] - f[i]) / h for i in range(N)]
u_3 = [6*(u_2[i + 1] - u_2[i])/ (2*h) for i in range(N-1)]

for i in range(N-1):
  for j in range(N-1):
    if i == j:
      A[i][j] = 2
      if j + 1 < (N-1):
        A[i][j + 1] = 1/2
      if i + 1 < (N-1):
        A[i + 1][j] = 1/2

c = np.linalg.solve(A, u_3)
c = np.append(c,0)

a = f[1:]

d = np.array([c[0]/h])
for i in range(1,N):
  d = np.append(d, (c[i] - c[i-1])/h)
  
b = np.array([u_2[0] + c[0]*h/3])
for i in range(1,N):
  b = np.append(b, ((c[i-1] + 2*c[i])*h/6)+ u_2[i])


In [107]:
f_z = a[-1] + b[-1]*(z-x[-1]) + (c[-1] * (z - x[-1])**2)/2 + (d[-1] * (z - x[-1])**3)/6
print(f'f({z}): {np.round(f_z,3)}')


f(0.95): 0.312


In [115]:
delta_metod_sp = (0.1 * (h/2)**4)/24
delta_v_sp = (2**(N+1)) * 0.001/(np.e * N * np.log(N))
print(f'delta_метода {delta_metod_sp}')
print(f'delta: {np.round(delta_metod_sp + delta_v_sp, 3)}')

delta_вычисл 2.6041666666666673e-08
delta: 0.033


Таким образом, в пределах погрешности интерполяционные значения функции совпадают. В обоих методах основной вклад в ошибку вносит вычислительная погрешность, при этом погрешность метода сплайнами меньше.